In [1]:
!pwd

/home/20203068


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from openai import OpenAI
import re

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# OpenAI GPT-4o 키
client = OpenAI(api_key="sMA") 

# 감정 라벨
emotion_labels = ["기쁨", "슬픔", "외로움", "두려움", "평온", "설렘", "신남", "분노"]

# STT 모델 (Whisper) 로드 
stt_model_id = "openai/whisper-small"
stt_model = AutoModelForSpeechSeq2Seq.from_pretrained(stt_model_id).to(device)
stt_processor = AutoProcessor.from_pretrained(stt_model_id)

stt_pipe = pipeline(
    "automatic-speech-recognition",
    model=stt_model,
    tokenizer=stt_processor.tokenizer,
    feature_extractor=stt_processor.feature_extractor,
    device=device,
)

# 대화 상태 저장 
chat_history = [
    {"role": "system", "content": "너는 독거노인과 따뜻하게 대화하는 친구야. 질문도 하면서 친절하게 말해줘."}
]

# GPT 대화 응답 함수
def generate_chat_response(chat_history):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=chat_history,
        max_tokens=500,
        temperature=0.7,
    )
    return response.choices[0].message.content

# 감정 분석 함수
def analyze_emotion_from_history(chat_history):
    messages = chat_history + [{
        "role": "system",
        "content": "지금까지의 사용자와 AI 간의 전체 대화를 바탕으로 감정을 하나로 분석해줘. 감정: " + ", ".join(emotion_labels) + " 중 하나로 '감정: [감정]' 형식으로 말해줘."
    }]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=300,
        temperature=0.7,
    )

    gpt_reply = response.choices[0].message.content
    emotion_match = re.search(r"감정:\s*(\w+)", gpt_reply)
    emotion = emotion_match.group(1) if emotion_match else "분석 실패"
    return emotion


Device set to use cuda:0


In [3]:
def stt_to_ttr(audio_file):
    stt_result = stt_pipe(audio_file)
    user_input = stt_result["text"]
    print("Whisper 결과:", user_input)

    chat_history.append({"role": "user", "content": user_input})
    reply = generate_chat_response(chat_history)
    chat_history.append({"role": "assistant", "content": reply})

    print("GPT 응답:", reply)

# # 테스트 실행 
# audio_file = "Gyeongsangdo/wavs/say_set1_collectorgs25_speakergs2215_15_0_16_3.wav"
# stt_to_ttr(audio_file)

# # 감정 분석
# final_emotion = analyze_emotion_from_history(chat_history)
# print(f"전체 대화 감정 분석 결과: {final_emotion}")

# 대화 1
stt_to_ttr("Gyeongsangdo/wavs/say_set2_collectorgs131_speakergs1126_32_0_43_1.wav")

# 대화 2
stt_to_ttr("Gyeongsangdo/wavs/say_set2_collectorgs131_speakergs1126_32_0_43_2.wav")

# 대화 3
stt_to_ttr("Gyeongsangdo/wavs/say_set2_collectorgs131_speakergs1126_32_0_43_4.wav")

# 대화 로그 확인
print(" 현재 chat_history 상태:")
for msg in chat_history:
    print(f"{msg['role']}: {msg['content']}")

# 감정 분석
final_emotion = analyze_emotion_from_history(chat_history)
print(f" 전체 대화 감정 분석 결과: {final_emotion}")

/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Whisper 결과:  태풍에 대한 기억은 옛날에 매일 민간 올 때 제가 울렁도 있었거든요.
GPT 응답: 울렁도에 사셨군요! 그곳은 정말 아름다운 섬으로 유명하죠. 태풍이 올 때면 바람과 파도가 정말 무섭고 위험했을 것 같아요. 그 시절에는 어떻게 대처하셨나요? 가족이나 이웃들과 함께 대피하셨나요?


/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Whisper 결과:  그 이틀에도 우리가 나오지 못했어요
GPT 응답: 정말 힘든 시간이었겠어요. 이틀 동안 집 안에만 계셔야 했다니 불안하고 답답하셨을 것 같아요. 그때는 가족들이랑 어떻게 시간을 보내셨나요? 혹시 그 경험을 통해 배우신 점이나 특별한 기억이 있으신가요?


/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Whisper 결과:  그 때 호텔에서 이렇게 내다보는데 얼마나 태풍이 불면 차가 막 뒤로 움직이더라고
GPT 응답: 정말 무서운 경험이셨겠어요. 차가 뒤로 움직일 정도면 바람이 아주 강했을 텐데, 안전한 곳에 계셔서 다행이에요. 그런 상황에서는 아무래도 마음이 많이 불안했을 텐데, 그때 어떻게 마음을 가라앉히셨나요? 혹시 주변에 같이 계셨던 분들과 서로 의지하셨나요?
 현재 chat_history 상태:
system: 너는 독거노인과 따뜻하게 대화하는 친구야. 질문도 하면서 친절하게 말해줘.
user:  태풍에 대한 기억은 옛날에 매일 민간 올 때 제가 울렁도 있었거든요.
assistant: 울렁도에 사셨군요! 그곳은 정말 아름다운 섬으로 유명하죠. 태풍이 올 때면 바람과 파도가 정말 무섭고 위험했을 것 같아요. 그 시절에는 어떻게 대처하셨나요? 가족이나 이웃들과 함께 대피하셨나요?
user:  그 이틀에도 우리가 나오지 못했어요
assistant: 정말 힘든 시간이었겠어요. 이틀 동안 집 안에만 계셔야 했다니 불안하고 답답하셨을 것 같아요. 그때는 가족들이랑 어떻게 시간을 보내셨나요? 혹시 그 경험을 통해 배우신 점이나 특별한 기억이 있으신가요?
user:  그 때 호텔에서 이렇게 내다보는데 얼마나 태풍이 불면 차가 막 뒤로 움직이더라고
assistant: 정말 무서운 경험이셨겠어요. 차가 뒤로 움직일 정도면 바람이 아주 강했을 텐데, 안전한 곳에 계셔서 다행이에요. 그런 상황에서는 아무래도 마음이 많이 불안했을 텐데, 그때 어떻게 마음을 가라앉히셨나요? 혹시 주변에 같이 계셨던 분들과 서로 의지하셨나요?
 전체 대화 감정 분석 결과: 두려움
